In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install flask
!pip install pyngrok
!pip install transformers
!pip install streamlit

In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from pyngrok import ngrok
import re

# Initialize Flask application
app = Flask(__name__)

# Set the model name
model_name = "tito92/finance_finetune_model"

# Cache the model and tokenizer loading
def load_model_and_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

# Load the model and tokenizer
model, tokenizer = load_model_and_tokenizer(model_name)

# Function to generate text based on input prompt
def generate_text(prompt, max_length=100):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length)
    result = pipe(f"<s>[INST] {prompt} [/INST]")[0]['generated_text']
    # Remove <s>[INST] ... [/INST] from the result
    result = re.sub(r"<s>\[INST\].*?\[/INST\]", "", result).strip()
    return result

# Route for generating text based on input prompt
@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    prompt = data.get('prompt')
    response_text = generate_text(prompt)
    return jsonify({'generated_text': response_text})

if __name__ == '__main__':
    try:
        # Start ngrok and get the public URL
        ngrok.set_auth_token('2aTkM8Ork8kAzWMH1MoLSICaPQL_5kcYe3ooHkYpxpjWtTvm7')
        http_tunnel = ngrok.connect(addr='5000', proto='http')
        print(f"Public URL: {http_tunnel.public_url}")
        
        # Run the Flask app
        app.run(host='0.0.0.0', port=5000)
    
    except Exception as e:
        print(f"Error starting ngrok or Flask application: {str(e)}")


2024-07-14 14:30:23.996098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 14:30:23.996198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 14:30:24.104687: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Public URL: https://7836-34-171-235-240.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
